In [ ]:
from google.colab import files
uploaded = files.upload()

Saving AB_US_2020.csv to AB_US_2020.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from math import sqrt
from sklearn.model_selection import  GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from joblib import  dump, load
import plotly.express as px


In [ ]:
# reading from my local computer
df = pd.read_csv("/content/AB_US_2020.csv")

## The link for the github repository where the airbnb data is found is: 
https://github.com/richardOlson/resfeber_data

In [ ]:
# reading from my github
df = pd.read_csv("https://raw.githubusercontent.com/richardOlson/resfeber_data/main/data/AB_US_2020.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.



# Read  comments below to understand which model was used in the airbnb predict function.

In [ ]:
# The model that is being used from this notebook is the one from the gradient boost, that was 
# trained on the data that has the outliers removed  (called no_outlier).  It was also trained on only 
# the data that had one night in the number of nights.  The model used is down near the end of this notebook

In [ ]:
df.head(3)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city
0,38585,Charming Victorian home - twin beds + breakfast,165529,Evelyne,NaN,28804,35.65146,-82.62792,Private room,60,1,138,16/02/20,1.14,1,0,Asheville
1,80905,French Chic Loft,427027,Celeste,NaN,28801,35.59779,-82.55540,Entire home/apt,470,1,114,07/09/20,1.03,11,288,Asheville
2,108061,Walk to stores/parks/downtown. Fenced yard/Pet...,320564,Lisa,NaN,28801,35.60670,-82.55563,Entire home/apt,75,30,89,30/11/19,0.81,2,298,Asheville


In [ ]:

df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
city                               object
dtype: object

In [ ]:
print(f"The shape of the the dataset is {df.shape}")
df.isnull().sum()

The shape of the the dataset is (226030, 17)


id                                     0
name                                  28
host_id                                0
host_name                             33
neighbourhood_group               115845
neighbourhood                          0
latitude                               0
longitude                              0
room_type                              0
price                                  0
minimum_nights                         0
number_of_reviews                      0
last_review                        48602
reviews_per_month                  48602
calculated_host_listings_count         0
availability_365                       0
city                                   0
dtype: int64

In [ ]:
df['minimum_nights'].unique()

array([        1,        30,         7,         3,        28,        90,
               4,         2,        21,        29,        14,        60,
             180,       120,        27,         5,         6,        20,
              25,        26,        31,       365,        32,        15,
              10,       360,        45,       100,        19,        68,
              83,       183,        99,        12,       240,       150,
             175,         9,        18,        43,       168,         8,
              13,       186,       300,       500,        76,        59,
              93,       200,       110,        40,       182,       290,
             999,        91,        56,        33,      1000,        35,
             600,       153,        92,       400,       160,        50,
             271,        55,       364,        11,        95,       210,
              16,        23,        80,        89,        85,       118,
              22,        75,       187,        36, 

In [ ]:
# looking at those prices that are less than 19 dollars
cheap = df[df['price'] < 19]
print(cheap.shape)
print(cheap["price"].value_counts())

px.histogram(data_frame=cheap, x='price')

(454, 17)
18    81
10    75
15    71
17    63
0     62
16    61
14    12
12    11
13     5
1      5
11     4
5      3
9      1
Name: price, dtype: int64


In [ ]:
# looking where those places are that have the 2000 dollar per night
expensive = df[df["price"] > 1930]
print(expensive.shape)
expensive.tail(3)

(1945, 17)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city
225143,42232379,"2021 Presidential Inauguration, 1/17 - 1/21, 5...",326230731,Gabriela,NaN,"Dupont Circle, Connecticut Avenue/K Street",38.90396,-77.04056,Entire home/apt,2000,4,0,NaN,NaN,4,5,Washington D.C.
225144,42232571,"2021 Presidential Inauguration, 1/17 - 1/21, ...",326230731,Gabriela,NaN,"Dupont Circle, Connecticut Avenue/K Street",38.90428,-77.03946,Entire home/apt,2000,4,0,NaN,NaN,4,5,Washington D.C.
225779,45079632,"INAUGURATION Luxury, walk to Capitol, museums",62733,Adolph,NaN,"Downtown, Chinatown, Penn Quarters, Mount Vern...",38.90385,-77.01701,Entire home/apt,2000,2,1,07/09/20,1.0,2,55,Washington D.C.


In [ ]:
# histogram of the price distibution
px.histogram(data_frame=df['price'], x="price")

In [ ]:
# looking to see what the valuse of the prices will be
# to possibly remove the outliers from the list
print(f"this the the standard deviation for the data {df['price'].std()}")
print(f"this is the mean for the data: {df['price'].mean()}")

this the the standard deviation for the data 570.3536091806417
this is the mean for the data: 219.71652877936558


In [ ]:
# seeing what the cut is for 3 std from the mean on both sides
low_cut = df['price'].mean() - (df['price'].std() * 3)
high_cut = df['price'].mean() + (df['price'].std() * 3)
low_cut, high_cut

(-1491.3442987625594, 1930.7773563212904)

In [ ]:
# further down I will cut the data and compare the results with not 
# removing the outliers.
# I will remove those data points that are less than 15 dollars and down, on the low end.
# On the high end I will remove those points of data that are greater than 1930 and up.

In [ ]:
lots_of_night = df[df['minimum_nights'] == 100000000]
#lots_nights_df = df[night_mask]
lots_of_night

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city
196520,15344978,Large Room in Noe/Mission 1br w/private entrance,470,Josh,NaN,Noe Valley,37.75071,-122.42521,Private room,68,100000000,8,06/11/17,0.17,1,0,San Francisco


In [ ]:
df['minimum_nights'].value_counts()

1      60427
2      54748
30     33532
3      30931
4       9513
       ...  
49         1
345        1
229        1
102        1
456        1
Name: minimum_nights, Length: 169, dtype: int64

In [ ]:
df["room_type"].unique()

array(['Private room', 'Entire home/apt', 'Hotel room', 'Shared room'],
      dtype=object)

In [ ]:
df['room_type'].value_counts()

Entire home/apt    154173
Private room        65887
Shared room          4029
Hotel room           1941
Name: room_type, dtype: int64

In [ ]:
df['price'].unique().sum

array([  60,  470,   75, ..., 1134, 1938, 1109])

In [ ]:
# We can probably drop the neighbourhood_group because we already have the zip and the lat and lon
arr = df["neighbourhood_group"].unique()
print(f"The number of unique values is: {len(arr)}")
arr

The number of unique values is: 35


array([nan, 'Hawaii', 'Kauai', 'Maui', 'Honolulu', 'Other Cities',
       'City of Los Angeles', 'Unincorporated Areas', 'Manhattan',
       'Brooklyn', 'Queens', 'Staten Island', 'Bronx', 'Providence',
       'Washington', 'Newport', 'Bristol', 'Kent', 'Central Area',
       'Other neighborhoods', 'West Seattle', 'Downtown', 'Ballard',
       'Capitol Hill', 'Beacon Hill', 'Seward Park', 'Queen Anne',
       'Rainier Valley', 'Lake City', 'Cascade', 'Delridge',
       'University District', 'Northgate', 'Magnolia', 'Interbay'],
      dtype=object)

In [ ]:
# making a copy of the df
df = dropped_outlier.copy()

In [ ]:
dropped_outlier = df[df['minimum_nights'] != 100000000]
dropped_outlier.shape

(226029, 17)

In [ ]:
num_nights = [1]
# filtering out all the nights that are not just the 1 minimum night
df = df[df["minimum_nights"].isin(num_nights)]
df.shape

(60427, 17)

In [ ]:
# dropping the columns
cols_to_keep = [
             "latitude",
              "longitude",
              "room_type",
             # "minimum_nights" -- these have been filtered out by just using the 1 night
]
Y_val = ["price"]
Y = df[Y_val]

X = df[cols_to_keep]
X.shape, Y.shape

((60427, 3), (60427, 1))

In [ ]:
# making the room_type as a numerical value
def room_to_num(room:str):
  # the string is checked
  if room == "Private room":
    return 0
  if room == "Entire home/apt":
    return 1
  if room == "Hotel room":
    return 2
  if room == "Shared room":
    return 3


In [ ]:
X = X.copy()
X['room_type'] = X['room_type'].apply(room_to_num, )
X.head(3)

,latitude,longitude,room_type
0,35.65146,-82.62792,0
1,35.59779,-82.55540,1
3,35.57864,-82.59578,1


In [ ]:
# splitting the data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.1, 
                                                    random_state=42)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((54384, 3), (54384, 1), (6043, 3), (6043, 1))

In [ ]:
# doing the same thing with Nuerual network
# need to the do the clean function which will remove the features that aren't used
# will use the scaler to scaler the data
# Will spit out the X_train, X_test and the y_train and y_test



In [ ]:
# running the function above on the data
x_train, x_test, y_train, y_test = prepare_dataframe(theDf=df, split_train_test=True)

In [ ]:
# looking at the size of the data
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((54384, 4), (6043, 4), (54384, 1), (6043, 1))

In [ ]:
# doing a random forrest regressor
m_forrest = RandomForestRegressor(max_depth=5, random_state=42)
y_train_raveled = y_train.values.ravel()
m_forrest.fit(x_train, y_train_raveled)
pred_test = m_forrest.predict(x_test)
print(f"This the r score of the test is {m_forrest.score(x_test, y_test.values.ravel())}")
pred_train = m_forrest.predict(x_train)
print(f"This is the r squared score for the training {m_forrest.score(x_train, y_train_raveled)}")


This the r score of the test is 0.006166803200832849
This is the r squared score for the training 0.1151300157599847


In [ ]:
lin_mod = LinearRegression()
lin_mod.fit(x_train, y_train)
# doing the prediction with the test set
pred_test = lin_mod.predict(x_test)
# Getting the coeficient 
the_coef = lin_mod.coef_
# Getting the mean squared error
mean_squared_err = mean_squared_error(y_test, pred_test)
the_r_score = r2_score(y_test, pred_test)

print(f"The coeficients for the linear regression are  {the_coef}")
print(f"The root mean squared error is {sqrt(mean_squared_err):.2f}")
print(f"The r score is {the_r_score:.2f}")
# doing the plotting
#plt.scatter(x_test, y_test, color="blue")
#plt.plot(x_test, pred_test, color="red", linewidth=3)
#plt.show()

The coeficients for the linear regression are  [[-2.61323438 -0.58594377 59.82924621]]
The root mean squared error is 565.15
The r score is 0.01


In [ ]:
# this is the baseline for the y_train data of the price for just 1 night
y_mean = y_train.mean()
print(f"This the the mean of the y train {y_mean}")
# making a numpy array that has the values of the mean filled in
a = np.full(y_train.shape, y_mean)

# finding what would be the root mean squared error if we just picked the mean only
print(f"This is the rmse if we just picked the mean of the training data every time {sqrt(mean_squared_error(y_train, a)):.2f}")

This the the mean of the y train price    206.814798
dtype: float64
This is the rmse if we just picked the mean of the training data every time 628.96


In [ ]:
type(y_test)

pandas.core.frame.DataFrame

In [ ]:
# doing a random forrest regressor
m_forrest = RandomForestRegressor(max_depth=5, random_state=42)
m_forrest.fit(x_train, y_train.values.ravel())
pred_test = m_forrest.predict(x_test)
print(f"This the r score of the test is {m_forrest.score(x_test, y_test.values.ravel())}")
pred_train = m_forrest.predict(x_train)
print(f"This is the r squared score for the training {m_forrest.score(x_train, y_train.values.ravel())}")
print(f"This the rmse for the random forrest for the test data {sqrt(mean_squared_error(y_test, pred_test)):.2f}")
print(f"This the rmse for the random forrest for the training data {sqrt(mean_squared_error(y_train, pred_train)):.2f}")

This the r score of the test is 0.006166803200832849
This is the r squared score for the training 0.1151300157599847
This the rmse for the random forrest for the test data 565.25
This the rmse for the random forrest for the training data 591.65


In [ ]:
# will try to do some tuning of the model to get a better score on the test

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.layers import  Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# making the room_type as a numerical value
def room_to_num(room:str):
  # the string is checked
  if room == "Private room":
    return 0
  if room == "Entire home/apt":
    return 1
  if room == "Hotel room":
    return 2
  if room == "Shared room":
    return 3

def prepare_dataframe(theDf, split_train_test=False, num_nights=None, scale=False):
  """
  This is the function that will get the data ready for the 
  neural network.

  num_nights:   is the min_num_nights that you want to keep in the dataframe
  If is left with None, then one night only will be used
  num_nights should be an int in a list.

  Scale:  This is used if it is wanted to have the data scaled.
          The standard scaler is used.
  """
  # if the number of nights are not supplied, then 
  # the program will slim it down to just 1 night
  
  if num_nights == None: 
    num_nights = [1]
  
  # keeping the num nights that you need to keep
  df = theDf[theDf['minimum_nights'].isin(num_nights)]
  # removing the outliers
  df = df[df["minimum_nights"] != 100000000]
  

  features = ["latitude",
              "longitude",
              "room_type",
              "minimum_nights"]
  target = ['price']

  # making the slimmed down dataframe
  X = df[features]
  y = df[target]
  
  X = X.copy()
  # making the room to a number value
  X['room_type'] = X["room_type"].apply(room_to_num)
  if scale == True:
    # doing the scaling of the data
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
  if split_train_test == True:
    # doing the split of the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=42)

    return X_train, X_test, y_train, y_test 
  else:
    return X, y
  


In [ ]:
df.shape

(60427, 17)

In [ ]:
# making the data for the neural network
x_train, x_test, y_train, y_test = prepare_dataframe(df, split_train_test=True, scale=True)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((54384, 4), (6043, 4), (54384, 1), (6043, 1))

In [ ]:
# looking to see what the data will look like in the first row
x_train[:1]

array([[ 0.23231635, -0.76475817, -0.98177349,  0.        ]])

In [ ]:
the_input = Input(shape=(4,))

# building the model 
j = Dense(64, activation="elu", name="first_layer")(the_input)
j = Dense(16, activation="elu", name="second_layer")(j)
j = Dense(16, activation="elu", name="third")(j)
j = Dense(8, activation="elu", name="fourth")(j)

output = Dense(1, activation="linear")(j)

# making the model
model = Model(the_input, output)

# doing the compilation of the model
model.compile(optimizer="adam", loss='mse', metrics=["accuracy"])
# making it to stop early if needed
earlyStop = EarlyStopping(monitor='loss', patience=3)

model.summary()

# doing the fitting of the model
model.fit(x=x_train, y=y_train, batch_size=16, epochs=20, callbacks=earlyStop, validation_data=(x_test, y_test))


Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
first_layer (Dense)          (None, 64)                320       
_________________________________________________________________
second_layer (Dense)         (None, 16)                1040      
_________________________________________________________________
third (Dense)                (None, 16)                272       
_________________________________________________________________
fourth (Dense)               (None, 8)                 136       
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 9         
Total params: 1,777
Trainable params: 1,777
Non-trainable params: 0
___________________________________________________

In [ ]:
# going to do the neural network again with 1 night and also 2 nights
# and see if this makes a difference


In [ ]:
the_input = Input(shape=(4,))

# building the model 
j = Dense(64, activation="elu", name="first_layer")(the_input)
j = Dense(64, activation="elu", )(j)
j = Dense(64, activation="elu", )(j)

output = Dense(1, activation="linear")(j)

# making the model
model = Model(the_input, output)

# doing the compilation of the model
model.compile(optimizer="adam", loss='mse', metrics=["accuracy"])
# making it to stop early if needed
earlyStop = EarlyStopping(monitor='loss', patience=3)

model.summary()

# doing the fitting of the model
model.fit(x=x_train, y=y_train, batch_size=16, epochs=20, callbacks=earlyStop, validation_data=(x_test, y_test))


Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
first_layer (Dense)          (None, 64)                320       
_________________________________________________________________
dense_26 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_27 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 65        
Total params: 8,705
Trainable params: 8,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
3399/3399 [==============================] - 7s 2ms/step - loss: 391742.8438 - accuracy: 5.5163e-05 - val_l

In [ ]:
x_train, x_test, y_train, y_test = prepare_dataframe(df, split_train_test=True, scale=False) 
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((54384, 4), (6043, 4), (54384, 1), (6043, 1))

In [ ]:
# looking at the first row of the data
x_train[:1]

,latitude,longitude,room_type,minimum_nights
204618,37.15015,-122.15256,0,1


In [ ]:
# doing a gradient boost model
# will be using grid search to find some of the best parameters
# will create the hyperparameters that will be used in the modle

# creating the pipeline which is just the model in the pipeline
pipe = make_pipeline(GradientBoostingRegressor(n_estimators =1000, random_state=42, verbose=1, n_iter_no_change=50,))


params = {
    "gradientboostingregressor__learning_rate": [.1, .05, .08],
    "gradientboostingregressor__min_samples_leaf": [2,5, 7],
    "gradientboostingregressor__max_depth": [   10 , 13],

}

In [ ]:
# doing with 2 nights to see what is the best
x_train2, x_test2, y_train2, y_test2 =prepare_dataframe(no_outliers, split_train_test=True, num_nights=[1,2])
x_train2.shape, x_test2.shape, y_train2.shape, y_test2.shape

((102832, 4), (11426, 4), (102832, 1), (11426, 1))

In [ ]:
# Running the gridsearch with now 1 and 2 nights 
m_grid = GridSearchCV(estimator=pipe, param_grid=params, n_jobs=-1, verbose=1,scoring="neg_root_mean_squared_error" )
# doing the fitting of the model
m_grid.fit(x_train2, y_train2.values.ravel())

# Printing out the best parameters and the the best score this time
print(f"The best params were: {m_grid.best_params_}")
print(f"The best score was: {-m_grid.best_score_}")

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 39.4min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 77.8min finished


      Iter       Train Loss   Remaining Time 
         1       35644.6096            4.93m
         2       34759.3346            4.97m
         3       33961.2649            4.95m
         4       33196.7634            4.94m
         5       32533.8155            4.93m
         6       31912.8479            4.91m
         7       31333.9164            4.91m
         8       30766.2728            4.91m
         9       30253.3303            4.91m
        10       29782.3286            4.90m
        20       26188.0963            4.81m
        30       24310.4264            4.73m
        40       23164.9801            4.69m
        50       22538.5523            4.69m
        60       22164.5507            4.67m
        70       21753.1876            4.64m
        80       21470.7178            4.61m
        90       21137.7456            4.58m
       100       20685.8910            4.56m
       200       18870.3001            4.24m
The best params were: {'gradientboostingregressor__lea

In [ ]:
# will now load the model and try it on one row of data
mod = load("gradient_boost_model.joblib")
print(mod.predict(pd.DataFrame({"lat": 37.15015, "lon":-122.15256	, "room_type":0, "num_nights":1}, index=[0])))

[130.33040642]


In [ ]:
# using the grid_search
m_grid = GridSearchCV(estimator=pipe, param_grid=params, n_jobs=2, verbose=1, )
# doing the fitting of the model
m_grid.fit(x_train, y_train.values.ravel())

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  5.6min
[Parallel(n_jobs=2)]: Done 135 out of 135 | elapsed: 10.0min finished
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1      387031.6286            1.37m
         2      378085.9353            1.37m
         3      367042.7997            1.35m
         4      359635.4540            1.35m
         5      351177.1129            1.35m
         6      345054.1004            1.35m
         7      339432.5757            1.35m
         8      333521.9646            1.34m
         9      328137.1239            1.34m
        10      325156.8866            1.34m
        20      296337.1831            1.33m
        30      283952.8088            1.33m
        40      273309.8962            1.34m
        50      263159.3716            1.33m
        60      253721.2028            1.32m
        70      246616.4601            1.31m
        80      238362.7159            1.30m
        90      231451.9009            1.29m
       100      225602.2634            1.28m
       200      187023.1398            1.15m
       300      169138.8473            1.01m
       40

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('gradientboostingregressor',
                                        GradientBoostingRegressor(alpha=0.9,
                                                                  ccp_alpha=0.0,
                                                                  criterion='friedman_mse',
                                                                  init=None,
                                                                  learning_rate=0.1,
                                                                  loss='ls',
                                                                  max_depth=3,
                                                                  max_features=None,
                                                                  max_leaf_nodes=None,
                                                                  min_impurity_decrease=0.0,
              

In [ ]:
print(m_grid.best_params_)

{'gradientboostingregressor__learning_rate': 0.1, 'gradientboostingregressor__max_depth': 7, 'gradientboostingregressor__min_samples_leaf': 5}


In [ ]:
best = m_grid.best_estimator_

In [ ]:
print(m_grid.best_score_)

0.1866490814943767


In [ ]:
# doing the same thing again but using a different scorer
# using the grid_search
m_grid = GridSearchCV(estimator=pipe, param_grid=params, n_jobs=-1, verbose=1,scoring="neg_root_mean_squared_error" )
# doing the fitting of the model
m_grid.fit(x_train, y_train.values.ravel())

# Printing out the best parameters and the the best score this time
print(f"The best params were: {m_grid.best_params_}")
print(f"The best score was: {-m_grid.best_score_}")

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  8.7min finished


      Iter       Train Loss   Remaining Time 
         1      364923.6413            2.25m
         2      337864.6646            2.24m
         3      315157.5285            2.22m
         4      296894.6141            2.23m
         5      281393.0232            2.25m
         6      268277.0931            2.25m
         7      257575.4949            2.24m
         8      245984.3718            2.24m
         9      235920.0135            2.23m
        10      226111.8055            2.24m
        20      175351.7324            2.26m
        30      157308.6027            2.29m
        40      149050.1269            2.33m
        50      143761.2597            2.35m
        60      140101.6559            2.36m
The best params were: {'gradientboostingregressor__learning_rate': 0.1, 'gradientboostingregressor__max_depth': 13, 'gradientboostingregressor__min_samples_leaf': 5}
The best score was: 554.7748806262168


In [ ]:
# saving the model from the gradientboost
dump(m_grid.best_estimator_, "gradient_boost_model.joblib")

['gradient_boost_model.joblib']

In [ ]:
# loading the model back into the colab notebook
mod = load("gradient_boost_model.joblib")


In [ ]:
mod.get_params( )

{'gradientboostingregressor': GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                           init=None, learning_rate=0.1, loss='ls', max_depth=13,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=5, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=1000,
                           n_iter_no_change=50, presort='deprecated',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=1, warm_start=False),
 'gradientboostingregressor__alpha': 0.9,
 'gradientboostingregressor__ccp_alpha': 0.0,
 'gradientboostingregressor__criterion': 'friedman_mse',
 'gradientboostingregressor__init': None,
 'gradientboostingregressor__learning_rate': 0.1,
 'gradientboostingregressor__loss': 'ls',
 'gradientboosting

In [ ]:
# running gradient boost again and will first remove outliers from the data set before
# training the dataset.
# Will remove those data points that have a price of less than 15 dollars, and
# will remove those that have a price greater than 1930 dollars per night.


In [ ]:
no_outliers = df[(df['price'] >= 15) & (df['price'] <= 1930)]
no_outliers.shape

(223907, 17)

In [ ]:
# making sure that there are no outliers in the dataset
lDf = no_outliers[no_outliers["price"] < 15]
hDf = no_outliers[no_outliers["price"]> 1930]
lDf.shape , hDf.shape

((0, 17), (0, 17))

In [ ]:
# getting the new set with 1 night and train and then will do it for 2 nighs and then 
# train to see what will give the best model


In [ ]:
x_train, x_test, y_train, y_test = prepare_dataframe(no_outliers, split_train_test=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((53800, 4), (53800, 1), (5978, 4), (5978, 1))

In [ ]:
# running the model to find the best score

In [ ]:
# doing the same thing again but using a different scorer
# using the grid_search
m_grid = GridSearchCV(estimator=pipe, param_grid=params, n_jobs=-1, verbose=1,scoring="neg_root_mean_squared_error" )
# doing the fitting of the model
m_grid.fit(x_train, y_train.values.ravel())

# Printing out the best parameters and the the best score this time
print(f"The best params were: {m_grid.best_params_}")
print(f"The best score was: {-m_grid.best_score_}")

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed: 59.2min finished


      Iter       Train Loss   Remaining Time 
         1       37348.3188            1.79m
         2       36099.1633            1.79m
         3       34839.5205            1.80m
         4       33877.3608            1.80m
         5       32917.6398            1.80m
         6       32064.5262            1.81m
         7       31274.2119            1.81m
         8       30588.1202            1.80m
         9       29931.1747            1.79m
        10       29491.4710            1.79m
        20       26393.1791            1.78m
        30       24834.3216            1.79m
        40       23579.4498            1.79m
        50       22402.1462            1.77m
        60       21707.3755            1.77m
        70       21276.7639            1.77m
        80       20950.4644            1.77m
        90       20646.5681            1.76m
       100       20444.3709            1.75m
       200       18211.0646            1.60m
       300       16587.8607            1.41m
The best 

In [ ]:
# doing the dumping of the model  ---- This is the model that is being used in the 
# the airbnb prediction
dump(m_grid.best_estimator_, "gradient_boost_model.joblib")

['gradient_boost_model.joblib']